# Setup

## imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.api as sm
import scipy.stats as stats
import datetime
import warnings
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from IPython.display import display
from sklearn.preprocessing import StandardScaler, RobustScaler
from tqdm.auto import tqdm
import mapply
import warnings
import importlib

/home/rockhight/Documents/gitrepos/formacion_python/pandas/Ejercicio2/envi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Dataset

Los datos de origen son proporcionados en un archivos csv:

* udfs: dataset con datos de operaciones financieras.

### 2. Columnas y significado:

* nb: número de referencia de la operación.
* contract: identificador de contrato.
* udf_ref: identificador de operación de trading.
* fmly: familia a la que pertenece la operación financiera.
* grp: grupo al que pertenece la operación financiera.
* type: tipo de operación financiera.
* country: país de origen de la operación.
* udf_name: campo informado en el registro.
* num_value: valor numérico.
* string_value: valor de cadena de caracteres.
* date_value: valor de fecha.
* data_timestamp_part: marca temporal.
* data_date_part: fecha en la que se almacena la información.
* source_system: fuente de los datos.

### 3. Descripción del problema:

Si hacemos una visión general a nuestro conjunto de datos, podemos observar como hay hasta 10 registros (filas) para cada valor de *nb*, donde cada registro solo da información para un valor de *udf_name*. Esto es un gasto innecesario de almacenamiento y computación, además de complicar los futuros cálculos derivados de estos datos. Por esta razón, necesitamos convertir estos registros con el mismo *nb* a un solo registro.

Nuestro dataframe final tendrá que contener las siguientes columnas: `nb, M_CCY, M_CLIENT, M_CRDTCHRG, M_DIRECTIAV, M_DISCMARGIN, M_LIQDTYCHRG, M_MVA, M_RVA, M_SELLER, M_SUCURSAL`

* nb: debe contener el número de referencia de la operación.
* M_CLIENT, M_SELLER, M_CCY, M_SUCURSAL: deben mapear el valor de *string_value*
* M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA: deben mapear el valor de *num_value*


Una vez tengamos este resultado, necesitaremos eliminar las operaciones que no tengan informados ninguno de los siguientes campos:

M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

No informados en este caso significa que o son valores nulos, vacíos o 0, en el caso de los campos numéricos.

### 4. Reto:

* Obtener un dataframe final que contenga las columnas indicadas, con un registro por *nb* y con los valores correctos mapeados.
* Las operaciones con los campos M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA, M_SELLER no informados no deben existir.
* Hacerlo de la manera más eficiente posible a nivel computacional.

**NOTA:** Cada uno de los pasos descritos en el problema pueden efectuarse en una sola línea.

In [2]:
mapply.init(n_workers = -1)

In [3]:
%matplotlib inline

In [4]:
warnings.filterwarnings('ignore')

In [5]:
udfs = pd.read_csv("udfs.csv",sep = ';')
udfs[udfs['string_value'].notna()]['string_value'].unique()

array(['LB_TLECLER', '1999', 'CCMO', 'USD', 'EUR', 'AMAM', '5493',
       'PRUEBA', 'GBP', 'MXN', 'LB_VSTAVRE', 'WATT', 'SELLER 11',
       'CE_NGIRARD', 'SELLER 10', 'SOV_GSM', 'BRL', 'SSSSSSS', 'VVVVVV',
       '1212', 'MMMM', 'SELLER1', 'SELLER9'], dtype=object)

In [6]:
display(udfs.head())
display(udfs.info())

,nb,contract,udf_ref,fmly,grp,type,country,udf_name,num_value,string_value,date_value,data_timestamp_part,data_date_part,source_system
0,444444,3333,28786653,IRD,LN_BR,NaN,ESP,M_CCY,NaN,NaN,NaN,20201128041303,2020-12-30,Mx3EU
1,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_CRDTCHRG,30.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
2,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_SELLER,NaN,LB_TLECLER,NaN,20210203032054,2020-12-30,Mx3EU
3,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_LIQDTYCHRG,50.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
4,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_MVA,20.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nb                   386 non-null    int64  
 1   contract             386 non-null    int64  
 2   udf_ref              386 non-null    int64  
 3   fmly                 386 non-null    object 
 4   grp                  386 non-null    object 
 5   type                 241 non-null    object 
 6   country              386 non-null    object 
 7   udf_name             386 non-null    object 
 8   num_value            211 non-null    float64
 9   string_value         130 non-null    object 
 10  date_value           0 non-null      float64
 11  data_timestamp_part  386 non-null    int64  
 12  data_date_part       386 non-null    object 
 13  source_system        386 non-null    object 
dtypes: float64(2), int64(4), object(8)
memory usage: 42.3+ KB


None

In [7]:
df_pivot = udfs.pivot_table(index=['nb'], columns='udf_name', values=['string_value', 'num_value'], aggfunc='first')
df_pivot.head()

num_value                                                     
udf_name M_CRDTCHRG M_DIRECTIAV M_DISCMARGIN M_LIQDTYCHRG M_MVA M_RVA   
nb                                                                      
2222222        30.0         0.0         10.0         50.0  20.0   0.0  \
3815982         0.0         0.0          0.0          0.0   0.0   0.0   
8216817         NaN         0.0          NaN          NaN   NaN   NaN   
10000001       20.0         0.0         10.0         30.0   0.0   0.0   
10000009       20.0         0.0         10.0         30.0   0.0   0.0   

         string_value                                           
udf_name        M_CCY M_CLIENT M_PRUEBA    M_SELLER M_SUCURSAL  
nb                                                              
2222222           USD     CCMO      NaN  LB_TLECLER       1999  
3815982           NaN      NaN      NaN         NaN        NaN  
8216817           EUR      NaN      NaN        AMAM        NaN  
10000001          NaN      NaN      NaN     SELLER1       1999  
10000009          NaN      NaN      NaN     SELLER9       1999

In [8]:
df_pivot.columns = [col[-1] for col in df_pivot.columns.values]
df_pivot.reset_index(inplace=True)

In [9]:
'hola_mundo'.strip()

'hola_mundo'

In [10]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nb            61 non-null     int64  
 1   M_CRDTCHRG    30 non-null     float64
 2   M_DIRECTIAV   61 non-null     float64
 3   M_DISCMARGIN  30 non-null     float64
 4   M_LIQDTYCHRG  30 non-null     float64
 5   M_MVA         30 non-null     float64
 6   M_RVA         30 non-null     float64
 7   M_CCY         36 non-null     object 
 8   M_CLIENT      9 non-null      object 
 9   M_PRUEBA      1 non-null      object 
 10  M_SELLER      58 non-null     object 
 11  M_SUCURSAL    26 non-null     object 
dtypes: float64(6), int64(1), object(5)
memory usage: 5.8+ KB


In [11]:
df_filtered = df_pivot.dropna(subset=['M_DISCMARGIN', 'M_DIRECTIAV', 'M_LIQDTYCHRG', 'M_CRDTCHRG', 'M_MVA', 'M_RVA', 'M_SELLER'], how='all')
df_filtered.loc[:, ['M_DISCMARGIN', 'M_DIRECTIAV', 'M_LIQDTYCHRG', 'M_CRDTCHRG', 'M_MVA', 'M_RVA']] = df_filtered.loc[:, ['M_DISCMARGIN', 'M_DIRECTIAV', 'M_LIQDTYCHRG', 'M_CRDTCHRG', 'M_MVA', 'M_RVA']].replace(0, pd.NA)
df_filtered.dropna(subset=['M_DISCMARGIN', 'M_DIRECTIAV', 'M_LIQDTYCHRG', 'M_CRDTCHRG', 'M_MVA', 'M_RVA'], how='all', inplace=True)

In [12]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 0 to 60
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nb            27 non-null     int64 
 1   M_CRDTCHRG    17 non-null     object
 2   M_DIRECTIAV   13 non-null     object
 3   M_DISCMARGIN  17 non-null     object
 4   M_LIQDTYCHRG  15 non-null     object
 5   M_MVA         13 non-null     object
 6   M_RVA         7 non-null      object
 7   M_CCY         6 non-null      object
 8   M_CLIENT      9 non-null      object
 9   M_PRUEBA      1 non-null      object
 10  M_SELLER      25 non-null     object
 11  M_SUCURSAL    24 non-null     object
dtypes: int64(1), object(11)
memory usage: 2.7+ KB


In [14]:
df_filtered.to_csv("df_filtered.csv")